# Mask R-CNN - Train on Shapes Dataset


This notebook shows how to train Mask R-CNN on your own dataset. To keep things simple we use a synthetic dataset of shapes (squares, triangles, and circles) which enables fast training. You'd still need a GPU, though, because the network backbone is a Resnet101, which would be too slow to train on a CPU. On a GPU, you can start to get okay-ish results in a few minutes, and good results in less than an hour.

The code of the *Shapes* dataset is included below. It generates images on the fly, so it doesn't require downloading any data. And it can generate images of any size, so we pick a small image size to train faster. 

In [ ]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt

# Root directory of the project
ROOT_DIR = os.path.abspath("../../")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn.config import Config
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.model import log

from skimage.draw import rectangle

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

## Setup for Simplot GPU

In [ ]:
import pyodbc
import os
import pandas as pd

DATA_DIR = os.path.join(ROOT_DIR, "../data")

COCO_MODEL_PATH = os.path.join(DATA_DIR, "mask_rcnn_coco.h5")

# Enable all gpus
!export HIP_VISIBLE_DEVICES=0,1,2,3

ANNOTATION_COLS = ['XMax','XMin','YMin','YMax','LabelDescription']

passwd = os.environ['dgs_sandbox_pwd']

conn = pyodbc.connect('DSN=BIdevDatabase;'
                      'Database=Sandbox;'
                      'UID=DGSuser;'
                      'PWD=' + passwd)


# annotations_query = """SELECT a.ImageID, a.XMax, a.XMin, a.YMin, a.YMax, b.LabelDescription FROM [Sandbox].[kaggle].[Combined_Set_Detection_BBox] as a
# join [kaggle].[Class_Description] as b on b.LabelName = a.LabelName"""

class_descriptions = pd.read_sql("SELECT LabelName, LabelDescription from [kaggle].[Class_Description]",conn)

class_descriptions['LabelID'] = class_descriptions.index

display(class_descriptions.head())

#This now holds the annotation information.
bboxes = pd.read_sql("SELECT ImageID, XMax, XMin, YMin, YMax, LabelName FROM [Sandbox].[kaggle].[Combined_Set_Detection_BBox]", conn)

annotations = pd.merge(bboxes,class_descriptions, on='LabelName',how='inner')

display(annotations.head())

# This now holds the list of images
image_paths = pd.read_sql("SELECT ImageID, RelativePath, Height, Width, Mode from [kaggle].[Image_Path]", conn)

display(image_paths.head())

# Inner join on the two dataframes, so we now have images combined with associated annotations
annotated_image_paths = pd.merge(image_paths,annotations, on='ImageID',how='inner')

display(annotated_image_paths.head())

## Setup from offsite

In [ ]:
import pandas as pd

os.environ["CUDA_VISIBLE_DEVICES"]="-1"
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")

ANN_CACHE = '/home/thomas/Data/cache/'

class_descriptions = pd.read_parquet(ANN_CACHE + 'class_desc.parquet.gzip')

class_descriptions['LabelID'] = class_descriptions.index

display(class_descriptions.head())

#This now holds the annotation information.
bboxes = pd.read_parquet(ANN_CACHE + 'bboxes.parquet.gzip')

annotations = pd.merge(bboxes,class_descriptions, on='LabelName',how='inner')

display(annotations.head())

image_paths = pd.read_parquet(ANN_CACHE + 'image_paths.parquet.gzip')

display(image_paths.head())

##  Detect only certain classes ... for now

In [ ]:
#Only look at the apple class for now ... RESET INDEX IS IMPORTANT 
# since this classes uses array indexing a lot
classes_of_interest = class_descriptions.iloc[475:476].reset_index()

## Configurations

In [ ]:

class KaggleConfig(Config):
    
    NAME = "kaggle"

    #TODO experiment with this
    GPU_COUNT = 1
    IMAGES_PER_GPU = 2

    # Number of classes (including background)
    NUM_CLASSES = 1 + len(classes_of_interest) #1 + len(class_descriptions)  # + 1 for background

    #TODO experiment with this
    # Use small images for faster training. Set the limits of the small side
    # the large side, and that determines the image shape.
    IMAGE_MIN_DIM = 512
    IMAGE_MAX_DIM = 512

    # Use a small epoch since the data is simple
    STEPS_PER_EPOCH = 100
    
    # Skip detections with < 90% confidence
    DETECTION_MIN_CONFIDENCE = 0.75
    
config = KaggleConfig()
config.display()

## Utility Functions

In [ ]:
def get_ax(rows=1, cols=1, size=8):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Change the default size attribute to control the size
    of rendered images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

def images_with_classes(image_paths, classes):
    image_ids_with_desired_classes = bboxes[bboxes['LabelName'].isin(classes['LabelName'])]['ImageID'].unique()
    return image_paths[image_paths['ImageID'].isin(image_ids_with_desired_classes)]

## Subclass Utils.Dataset

In [ ]:
class KaggleImageDataset(utils.Dataset):

    #TODO better method documentation needed for this
    
    def load_kaggle_images(self, dataset_dir, image_paths, classes=class_descriptions):
        """Load a subset of the image dataset.
        dataset_dir: The root directory of the image dataset.
        classes: Dataframe : If provided, only loads images that have the given classes.
        """

        #Assumes class_descriptions, annotations, and bboxes are global DataFrames
        
        images = images_with_classes(image_paths,classes)
        
        #add this as a hack to allow a subset of classes
        #TODO can clean some stuff up if we are going to use all of the classes
        tmp_indices = {}

        # Add classes
        for i,row in classes.iterrows():
            tmp_indices[row['LabelID']] = i
            self.add_class("openImages", i, row['LabelDescription'])
            
        # Select down to the annotations instances for the desired classes
        salient_anns = annotations[annotations['LabelID'].isin(classes['LabelID'].values)]
        
        # Add images
        for i,row in images.iterrows():
            
            anns = salient_anns[salient_anns['ImageID'] == row['ImageID']].copy()
            
            #look up the new class id (which is used as an index, arghhh), add 1 since background is assigned 0
            anns['classID'] = anns.apply (lambda row: tmp_indices[row['LabelID']] + 1, axis=1)
            
            self.add_image(
                "openImages", image_id=i,
                path=os.path.join(dataset_dir, row['RelativePath']),
                width=row["Width"],
                height=row["Height"],
                annotations=anns)
            

    def load_mask(self, image_id):
        """Generate instance masks for an image.
       Returns:
        masks: A bool array of shape [height, width, instance count] with
            one mask per instance.
        class_ids: a 1D array of class IDs of the instance masks.
        """
        
        # Create rectangular bounding box since we are doing object detection, not segmentation
        # desired dimension is [height, width, instance_count]
        img = self.image_info[image_id]
        
        mask = np.zeros([img["height"], img["width"], len(img["annotations"])],
                        dtype=np.uint8)
        
        for i,(_,p) in enumerate(img["annotations"].iterrows()):
            # Create rectangular bounding box since we are doing object detection, not segmentation
    
            xmax = int(img["width"]*p['XMax'])
            xmin = int(img["width"]*p['XMin'])
            ymin = int(img["height"]*p['YMin'])
            ymax = int(img["height"]*p['YMax'])
            
            start = (ymin, xmin)  #top left corner ... are coordinates reversed?
            end = (ymax, xmax)  #height and width
            rr, cc = rectangle(start, end=end, shape=(img["height"],img["width"]))
            
            mask[rr, cc, i] = 1

        
        # Return mask, and array of class IDs of each instance.
        return mask.astype(np.bool), np.array(img['annotations']['classID'].values, dtype=np.int32)

    def image_reference(self, image_id):
        return self.image_info[image_id]['path']
    

## Load dataset and display samples

In [ ]:
ON_GPU_BOX = True

if ON_GPU_BOX:
    val_paths = image_paths[image_paths['RelativePath'].str.contains('validation',regex=False)]
    train_paths = image_paths[image_paths['RelativePath'].str.contains('train',regex=False)]
else:
    all_im = images_with_classes(image_paths, classes_of_interest)
    val_paths = all_im.iloc[0:5]
    train_paths = all_im.iloc[5:]
    DATA_DIR = '/home/thomas/Data/openImages/'


# Training dataset
dataset_train = KaggleImageDataset()
dataset_train.load_kaggle_images(DATA_DIR, train_paths, classes_of_interest)
dataset_train.prepare()

# Validation dataset
dataset_val = KaggleImageDataset()
dataset_val.load_kaggle_images(DATA_DIR, val_paths, classes_of_interest)
dataset_val.prepare()

In [ ]:
# Load and display random samples
image_ids = np.random.choice(dataset_train.image_ids, 10)
for image_id in image_ids:
    image = dataset_train.load_image(image_id)
    mask, class_ids = dataset_train.load_mask(image_id)
    visualize.display_top_masks(image, mask, class_ids, dataset_train.class_names)

## Create Model

In [ ]:
# Create model in training mode
model = modellib.MaskRCNN(mode="training", config=config,
                          model_dir=MODEL_DIR)

In [ ]:
# Which weights to start with?
init_with = "coco"  # imagenet, coco, or last

if init_with == "imagenet":
    model.load_weights(model.get_imagenet_weights(), by_name=True)
elif init_with == "coco":
    # Load weights trained on MS COCO, but skip layers that
    # are different due to the different number of classes
    # See README for instructions to download the COCO weights
    model.load_weights(COCO_MODEL_PATH, by_name=True,
                       exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", 
                                "mrcnn_bbox", "mrcnn_mask"])
elif init_with == "last":
    # Load the last model you trained and continue training
    model.load_weights(model.find_last(), by_name=True)

## Training

Train in two stages:
1. Only the heads. Here we're freezing all the backbone layers and training only the randomly initialized layers (i.e. the ones that we didn't use pre-trained weights from MS COCO). To train only the head layers, pass `layers='heads'` to the `train()` function.

2. Fine-tune all layers. For this simple example it's not necessary, but we're including it to show the process. Simply pass `layers="all` to train all layers.

In [ ]:
# Train the head branches
# Passing layers="heads" freezes all layers except the head
# layers. You can also pass a regular expression to select
# which layers to train by name pattern.
model.train(dataset_train, dataset_val, 
            learning_rate=config.LEARNING_RATE, 
            epochs=1, 
            layers='heads')

In [ ]:
# Fine tune all layers
# Passing layers="all" trains all layers. You can also 
# pass a regular expression to select which layers to
# train by name pattern.
model.train(dataset_train, dataset_val, 
            learning_rate=config.LEARNING_RATE / 10,
            epochs=2, 
            layers="all")

In [ ]:
# Save weights
# Typically not needed because callbacks save after every epoch
# Uncomment to save manually
# model_path = os.path.join(MODEL_DIR, "mask_rcnn_shapes.h5")
# model.keras_model.save_weights(model_path)

## Detection

In [ ]:
class InferenceConfig(ShapesConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

inference_config = InferenceConfig()

# Recreate the model in inference mode
model = modellib.MaskRCNN(mode="inference", 
                          config=inference_config,
                          model_dir=MODEL_DIR)

# Get path to saved weights
# Either set a specific path or find last trained weights
# model_path = os.path.join(ROOT_DIR, ".h5 file name here")
model_path = model.find_last()

# Load trained weights
print("Loading weights from ", model_path)
model.load_weights(model_path, by_name=True)

In [ ]:
# Test on a random image
image_id = random.choice(dataset_val.image_ids)
original_image, image_meta, gt_class_id, gt_bbox, gt_mask =\
    modellib.load_image_gt(dataset_val, inference_config, 
                           image_id, use_mini_mask=False)

log("original_image", original_image)
log("image_meta", image_meta)
log("gt_class_id", gt_class_id)
log("gt_bbox", gt_bbox)
log("gt_mask", gt_mask)

visualize.display_instances(original_image, gt_bbox, gt_mask, gt_class_id, 
                            dataset_train.class_names, figsize=(8, 8))

In [ ]:
results = model.detect([original_image], verbose=1)

r = results[0]
visualize.display_instances(original_image, r['rois'], r['masks'], r['class_ids'], 
                            dataset_val.class_names, r['scores'], ax=get_ax())

## Evaluation

In [ ]:
# Compute VOC-Style mAP @ IoU=0.5
# Running on 10 images. Increase for better accuracy.
image_ids = np.random.choice(dataset_val.image_ids, 10)
APs = []
for image_id in image_ids:
    # Load image and ground truth data
    image, image_meta, gt_class_id, gt_bbox, gt_mask =\
        modellib.load_image_gt(dataset_val, inference_config,
                               image_id, use_mini_mask=False)
    molded_images = np.expand_dims(modellib.mold_image(image, inference_config), 0)
    # Run object detection
    results = model.detect([image], verbose=0)
    r = results[0]
    # Compute AP
    AP, precisions, recalls, overlaps =\
        utils.compute_ap(gt_bbox, gt_class_id, gt_mask,
                         r["rois"], r["class_ids"], r["scores"], r['masks'])
    APs.append(AP)
    
print("mAP: ", np.mean(APs))